In [ ]:
!pip install transformers[torch] datasets evaluate sentencepiece tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00


In [ ]:
import json

In [ ]:
import transformers,torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,AutoModelForQuestionAnswering,T5ForConditionalGeneration
from transformers import T5Tokenizer, DataCollatorForSeq2Seq

In [ ]:
model_check='google/flan-t5-small'
model=T5ForConditionalGeneration.from_pretrained(model_check)
tokenizer=T5Tokenizer.from_pretrained(model_check)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install -U accelerate

In [ ]:
from datasets import load_dataset

In [ ]:
dataset=load_dataset('squad')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
fe='/content/data1.json'
with open(fe,'r') as file:
  dataset=json.load(file)

In [ ]:
dataset

[{'question': 'How early can autism be identified? What should parents do if they are concerned their young child may have autism?',
  'context': "Signs of Autism Spectrum Disorder (ASD) can be observed before the age of 1, and diagnosis by a specialist is possible from 18 months. However, many children are not diagnosed until after their 4th birthday due to factors like delayed assessment seeking and limited access to specialists. Less severe ASD cases and children from minority backgrounds often receive later diagnoses. Researchers are exploring methods for earlier diagnosis, such as eye tracking technologies. Diagnosis relies on clinical examination and additional information, as there's no single scan or blood test for autism. Parents are advised to trust their instincts regarding their child's development and consult their primary care pediatrician, who may then refer them to a specialist for a thorough evaluation.",
  'answer': "Parents are advised to trust their instincts about 

In [ ]:
questions=[]
context=[]
answers=[]
for item in dataset:
  questions.append(item['question'])
  context.append(item['context'])
  answers.append(item['answer'])

In [ ]:
context

In [ ]:
train_test_split = dataset["train"].train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
validation_dataset = train_test_split["test"]

In [ ]:
dataset_train = train_dataset.select(range(1000))

In [ ]:
dataset_valid=validation_dataset.select(range(500))

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
max_length=450
def preprocess_for_flan_t5(examples):
    inputs = ["question: " + question.strip() + " context: " + context for question, context in zip(examples["question"], examples["context"])]
    targets = [answer["text"][0] for answer in examples["answers"]]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length")

    decoder_start_token_id = tokenizer.pad_token_id

# Convert decoder_start_token_id to a tensor
    decoder_start_token_tensor = torch.tensor([decoder_start_token_id])

  model_inputs["decoder_input_ids"] = [torch.cat([decoder_start_token_tensor, label_ids[:-1]], dim=0) for label_ids in labels["input_ids"]]
    model_inputs["labels"] = labels["input_ids"]


In [ ]:
train_dataset = dataset_train.map(
    preprocess_for_flan_t5,
    batched=True,
    remove_columns=dataset["train"].column_names,
)
len(dataset["train"]), len(train_dataset)

(87599, 1000)

In [ ]:
sample=train_dataset[0]

In [ ]:
def preprocess_for_flan_t5_validation(examples):

    inputs = ["question: " + question.strip() + " context: " + context for question, context in zip(examples["question"], examples["context"])]


    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")


    example_ids = examples["id"]
    model_inputs["example_id"] = example_ids

    return model_inputs


In [ ]:
validation_dataset = dataset_valid.map(
    preprocess_for_flan_t5,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)
len(dataset["validation"]), len(validation_dataset)

(10570, 500)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
    num_rows: 1000
})

In [ ]:
validation_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
    num_rows: 500
})

In [ ]:
from collections import Counter
import string
import re
import numpy as np

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))

def compute_metrics(eval_preds):
    predictions, label_ids = eval_preds
    preds_flat = [pred.strip() for pred in predictions]
    labels_flat = [label.strip() for label in label_ids]

    f1 = np.mean([f1_score(pred, label) for pred, label in zip(preds_flat, labels_flat)])
    exact_match = np.mean([exact_match_score(pred, label) for pred, label in zip(preds_flat, labels_flat)])

    return {
        "exact_match": exact_match,
        "f1": f1,
    }


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer,Trainer

In [ ]:
model_name = model_check.split("/")[-1]


args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-squad",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=f"{model_name}-logs",
    logging_steps=50,

    fp16=False,
    push_to_hub=True,

)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator

)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.954200,2.078430
2,2.010200,1.410666
3,1.750700,1.221352


TrainOutput(global_step=375, training_loss=2.3567876892089843, metrics={'train_runtime': 296.16, 'train_samples_per_second': 10.13, 'train_steps_per_second': 1.266, 'total_flos': 490140979200000.0, 'train_loss': 2.3567876892089843, 'epoch': 3.0})

In [ ]:
import accelerate
accelerate.__version__

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Sabbasi-11/flan-t5-small-squad")
model = AutoModelForSeq2SeqLM.from_pretrained("Sabbasi-11/flan-t5-small-squad")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

ValueError: ignored

In [ ]:
context = "The Great Wall of China is one of the most famous structures in the world. It stretches over 13,000 miles and was originally built to protect the northern borders of the Chinese Empire from invading forces. The construction of the wall started as early as the 7th century BC, and it was built over several centuries by various dynasties. The most well-known sections of the wall were built during the Ming Dynasty. Today, the Great Wall is considered one of the New Seven Wonders of the World and remains a symbol of China's enduring strength and history."
question = "What was the primary purpose of building the Great Wall of China?"
input_text = f"question: {question} context: {context}"

In [ ]:
inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=450, truncation=True)

In [ ]:
outputs = model.generate(inputs, max_length=60, num_beams=6, early_stopping=False)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(answer)

protect the northern borders of the Chinese Empire from invading forces
